## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,41.00,87,75,8.05,broken clouds
1,1,Batagay,RU,67.6333,134.6333,34.65,79,100,6.64,overcast clouds
2,2,Portlethen,GB,57.0547,-2.1307,47.97,84,75,14.97,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,72.55,68,0,13.11,clear sky
4,4,Guerrero Negro,MX,27.9769,-114.0611,74.93,62,0,18.88,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 71
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prompt_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
prompt_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
3,3,Rikitea,PF,-23.1203,-134.9692,72.55,68,0,13.11,clear sky
4,4,Guerrero Negro,MX,27.9769,-114.0611,74.93,62,0,18.88,clear sky
5,5,Alyangula,AU,-13.8483,136.4192,76.75,86,54,5.59,broken clouds
6,6,Sibolga,ID,1.7427,98.7792,75.24,88,96,1.92,overcast clouds
11,11,Atuona,PF,-9.8000,-139.0333,76.37,72,56,19.10,broken clouds
13,13,Vaini,TO,-21.2000,-175.2000,78.96,83,20,4.61,few clouds
14,14,Puerto Ayora,EC,-0.7393,-90.3518,75.16,86,96,7.00,overcast clouds
16,16,Jamestown,US,42.0970,-79.2353,79.05,49,20,5.75,few clouds
17,17,Matay,EG,28.4190,30.7792,78.30,32,0,7.09,clear sky
21,21,Georgetown,MY,5.4112,100.3354,77.95,97,20,2.30,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
prompt_cities_df.count()

City_ID             184
City                184
Country             183
Lat                 184
Lng                 184
Max Temp            184
Humidity            184
Cloudiness          184
Wind Speed          184
City Description    184
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_prompt_cities_df = prompt_cities_df.dropna()
clean_prompt_cities_df.count()

City_ID             183
City                183
Country             183
Lat                 183
Lng                 183
Max Temp            183
Humidity            183
Cloudiness          183
Wind Speed          183
City Description    183
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = prompt_cities_df[["City", "Country", "Max Temp", "City Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
3,Rikitea,PF,72.55,clear sky,-23.1203,-134.9692,
4,Guerrero Negro,MX,74.93,clear sky,27.9769,-114.0611,
5,Alyangula,AU,76.75,broken clouds,-13.8483,136.4192,
6,Sibolga,ID,75.24,overcast clouds,1.7427,98.7792,
11,Atuona,PF,76.37,broken clouds,-9.8000,-139.0333,
13,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
14,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
16,Jamestown,US,79.05,few clouds,42.0970,-79.2353,
17,Matay,EG,78.30,clear sky,28.4190,30.7792,
21,Georgetown,MY,77.95,few clouds,5.4112,100.3354,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
3,Rikitea,PF,72.55,clear sky,-23.1203,-134.9692,People ThankYou
4,Guerrero Negro,MX,74.93,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
5,Alyangula,AU,76.75,broken clouds,-13.8483,136.4192,Groote Eylandt Lodge
6,Sibolga,ID,75.24,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
11,Atuona,PF,76.37,broken clouds,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
13,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Jamestown,US,79.05,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
17,Matay,EG,78.30,clear sky,28.4190,30.7792,Nag Aespen
21,Georgetown,MY,77.95,few clouds,5.4112,100.3354,Cititel Penang


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                183
Country             183
Max Temp            183
City Description    183
Lat                 183
Lng                 183
Hotel Name          183
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{City Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))